In [ ]:
import csv
import random
import classes
import crossover
import pandas as pd

from deap import base
from deap import creator
from deap import tools

In [ ]:
# Get data from csv file
def get_data(csv_file):
  data = []
  with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip first line (headers)
    for row in csv_reader:
      pos_name = row[0]
      latitude = float(row[1])
      longitude = float(row[2])
      address = row[3]
      entree_time = int(row[4])
      unloading_time = int(row[5])
      journey2pos_time = int(row[6])
      delivery_time = int(row[7])
      journey2unloadingpoint_time = int(row[8])
      checkout_time = int(row[9])
      min_travels = int(row[10])
      max_travels = int(row[11])
      extra_times = int(row[12])

      data.append((pos_name, latitude, longitude, address, 
        entree_time, unloading_time, journey2pos_time, delivery_time, journey2unloadingpoint_time, checkout_time,
        min_travels, max_travels, extra_times
      ))
  return data

In [ ]:
def shuffle_pos(pos):
  random.shuffle(pos)
  return pos

In [ ]:
points_of_sale = get_data('data.csv')
# Point_of_Sale list
pos = []
for point in points_of_sale:
  pos.append(classes.Point_of_Sale(point[0], point[1], point[2], point[3],
    point[4], point[5], point[6], point[7], point[8], point[9], 
    point[10], point[11], point[12]
  ))

In [ ]:
# weights is a tuple of -1.0s which means we want to Minimize two values: Distance and Time
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# individual's attribute sequence will shuffle a list of POS at creation
toolbox.register("sequence", shuffle_pos, pos)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.sequence)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
#https://deap.readthedocs.io/en/master/api/tools.html#id2
#https://en.wikipedia.org/wiki/Crossover_(genetic_algorithm)
#file:///C:/Users/llaaj/AppData/Local/Temp/MicrosoftEdgeDownloads/b9e8d235-e495-4ff4-86e8-6678782e06ba/download.pdf
toolbox.register("mate", crossover.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
# tournsize = The number of individuals participating in each tournament
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", crossover.evaluate)

In [ ]:
def main():
  mate_probability = 0.5
  mutation_probability = 0.2
  number_of_generations = 100
  population = toolbox.population(n=100)
  # Evaluate the entire population
  fitnesses = map(toolbox.evaluate, population)

  for individual, fitness in zip(population, fitnesses):
    individual.fitness.values = fitness

  for generation in range(number_of_generations):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Clone the selected individuals
    offspring = map(toolbox.clone, offspring)
    offspring = list(offspring)
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < mate_probability:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    for mutant in offspring:
      if random.random() < mutation_probability:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    
    population[:] = offspring

  return population  

In [ ]:
pop = main()

In [ ]:
pd.set_option('display.max_rows', None)
pop_df = pd.DataFrame(pop, columns=['POS1', 'POS2', 'POS3'])

fitnesses_list = []
for ind in pop:
  fitnesses_list.append(ind.fitness.values)

fitness_df = pd.DataFrame(fitnesses_list, columns=['Distance', 'Time'])

result_df = pd.concat([pop_df, fitness_df], axis=1)
result_df

In [ ]:

dict_df = pd.DataFrame(crossover.routes_responses_dict)
dict_df.transpose()